# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
!pip install -q promptlayer # openai API 조회 서비스

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [4]:
# !pip install pandas

In [5]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [6]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [13]:
from langchain.chat_models import ChatOpenAI, PromptLayerChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [19]:
# llm = ChatOpenAI(temperature=0.9)
llm = PromptLayerChatOpenAI(pl_tags=["langchain_3_230707"], temperature=0.9)


In [15]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [16]:
chain = LLMChain(llm=llm, prompt=prompt)

In [17]:
product = "Queen Size Sheet Set"
chain.run(product)

'RegalRest'

## SimpleSequentialChain

In [18]:
from langchain.chains import SimpleSequentialChain

In [28]:
# llm = ChatOpenAI(temperature=0.9)
llm = PromptLayerChatOpenAI(pl_tags=["langchain_3_230707"], temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)  # "{제품}을 만드는 회사를 설명하는 가장 좋은 이름은 무엇인가요?"

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [29]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description in Korean for the following \
    company:{company_name}"
)  # "다음 회사에 대한 한국어 설명을 20단어로 작성하세요:{company_name}"

# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [30]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [31]:
overall_simple_chain.run(product)



> Entering new  chain...
"Regal Comfort Beddings"
"리갈 컴포트 베딩은 편안한 수면을 위한 고급 침구 제품을 제공하는 회사입니다."

> Finished chain.


'"리갈 컴포트 베딩은 편안한 수면을 위한 고급 침구 제품을 제공하는 회사입니다."'

## SequentialChain

In [32]:
from langchain.chains import SequentialChain

In [39]:
# llm = ChatOpenAI(temperature=0.9)
llm = PromptLayerChatOpenAI(pl_tags=["langchain_230707"], temperature=0.9)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to Korean:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="Korean_Review"
)

In [40]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{Korean_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
)

In [41]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
)

In [42]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [43]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["Korean_Review", "summary","followup_message"],
    verbose=True
)

In [45]:
review = df.Review[5]
overall_chain(review)



> Entering new  chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'Korean_Review': '나는 그 맛이 별로라고 생각한다. 거품이 잘 유지되지 않고 이상하다. 상점에서 같은 제품을 구매했을 때는 훨씬 더 맛있었는데...\n오래된 제품이거나 가품일까!?',
 'summary': 'The reviewer believes that the taste is not good, the foam does not stay well, and wonders if the product is old or counterfeit since it was much tastier when purchased from the store before.',
 'followup_message': "Réponse: Nous sommes désolés d'apprendre que vous n'êtes pas satisfait du goût de notre produit et que la mousse ne tient pas bien. Nous tenons à vous assurer que nous fabriquons nos produits avec les meilleurs ingrédients et en suivant des procédés de qualité stricts. Il est possible que la différence de goût que vous avez notée soit due à des variations naturelles dans les ingrédients, mais nous comprenons votre déception. Nous vous recommandons de vérifier la date de péremp

## Router Chain

In [46]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [47]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [48]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [49]:
# llm = ChatOpenAI(temperature=0)
llm = PromptLayerChatOpenAI(pl_tags=["langchain_230707"], temperature=0.9)

In [50]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [51]:
destinations_str

'physics: Good for answering questions about physics\nmath: Good for answering math questions\nHistory: Good for answering history questions\ncomputer science: Good for answering computer science questions'

In [52]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [53]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [54]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [55]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
)

In [56]:
chain.run("What is black body radiation?")



> Entering new  chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


'Black body radiation refers to the phenomenon where an object, at a given temperature, emits electromagnetic radiation across a broad range of frequencies. A black body is an idealized object that fully absorbs all radiation incident upon it and completely emits radiation based on its temperature. It is called "black" because it absorbs all incident radiation without reflecting or transmitting any.\n\nWhen a black body is heated, it emits radiation in the form of photons. The intensity and distribution of this radiation depend on the temperature of the object. As the temperature increases, the emitted radiation spans a wider range of frequencies, starting from infrared and extending through visible light and up to ultraviolet and beyond.\n\nOne of the key aspects of black body radiation is that it follows a specific distribution called Planck\'s law, named after the physicist Max Planck who first derived it. Planck\'s law describes the intensity of the emitted radiation at different f

In [57]:
chain.run("what is 2 + 2")



> Entering new  chain...
math: {'input': '2 + 2'}
> Finished chain.


"Thank you for your kind words! I'm here to help with your math question. The expression 2 + 2 can be solved quite easily. Adding 2 and 2 together gives us a sum of 4. So, the answer to 2 + 2 is 4."

In [59]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new  chain...
None: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA (deoxyribonucleic acid) is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. It serves as a blueprint for building and maintaining the structures and functions of cells, tissues, and organs. DNA contains the genetic code that determines our traits, such as eye color, hair color, and susceptibility to certain diseases. Each cell needs DNA to carry out these important functions and to pass on the genetic information to the next generation of cells during cell division and reproduction.'

In [60]:
chain.run("how to make kimchi")



> Entering new  chain...
None: {'input': 'how to make kimchi'}
> Finished chain.


"To make kimchi, you will need the following ingredients:\n\n- 1 medium-sized Napa cabbage\n- 2 tablespoons sea salt\n- 1 tablespoon grated ginger\n- 4 cloves garlic, minced\n- 1 tablespoon sugar\n- 2 tablespoons fish sauce (optional for a non-vegetarian version)\n- 3 tablespoons Korean red pepper flakes (gochugaru)\n- 4 green onions, chopped\n- 1 medium-sized carrot, julienned\n\nHere is a step-by-step guide to making kimchi:\n\n1. Cut the Napa cabbage in half lengthwise and remove the core. Cut it into bite-sized pieces.\n\n2. Dissolve the sea salt in a bowl of water and soak the cabbage in it for about 2 hours. Flip the cabbage halfway through to ensure even salting.\n\n3. While the cabbage is soaking, make the kimchi paste by mixing the ginger, garlic, sugar, fish sauce (if using), and red pepper flakes in a separate bowl. Mix well to create a thick paste.\n\n4. After 2 hours, rinse the cabbage under cold water to remove excess salt. Drain and squeeze out any remaining water.\n\n5.